In [3]:
#NOAA Weather data download for Portland, Oregon

import requests
import pandas as pd

In [ ]:
NOAA_TOKEN = 'XXX'

In [6]:
#NOAA API Headers
headers = {'token': NOAA_TOKEN}

#Portland Airport Station ID (GHCND:USW00024229)
station_id = 'GHCND:USW00024229'

#Date range
start_date = '2025-01-01'
end_date = '2025-01-31'

datatype_ids = ['TMIN', 'TMAX', 'PRCP']

def fetch_noaa_data(station_id, datatype_ids, start_date,
                    end_date, token):
    results = []

    for dtype in datatype_ids:
        print(f"Fetching {dtype} data...")
        params = {
            'datasetid': 'GHCND',
            'stationid': station_id,
            'datatypeid': dtype,
            'startdate': start_date,
            'enddate': end_date,
            'units': 'metric',
            'limit': 1000
        }

        response = requests.get(
            'https://www.ncdc.noaa.gov/cdo-web/api/v2/data',
            headers={'token': token},
            params=params            
        )

        if response.status_code == 200:
            data = response.json().get('results', [])
            results.extend(data)
        else:
            print(f"Error fetching {dtype}: {response.status_code} - {response.text}")
        
    return pd.DataFrame(results)

raw_df = fetch_noaa_data(station_id, datatype_ids, start_date, end_date, NOAA_TOKEN)

if not raw_df.empty:
    raw_df['date'] = pd.to_datetime(raw_df['date']).dt.date

    df = raw_df.pivot_table(index='date', columns=
                            'datatype')
    
    if 'MIN' in df.columns: df['TMIN'] = df['TMIN']/10
    if 'MAX' in df.columns: df['TMAX'] = df['TMIN']/10
    if 'PRCP' in df.columns: df['PRCP'] = df['PRCP']/10

    print(df.head())

else:
    print("No data available")



Fetching TMIN data...
Fetching TMAX data...
Fetching PRCP data...
           value           
datatype    PRCP  TMAX TMIN
date                       
2025-01-01   0.3   6.7  4.4
2025-01-02  12.2   6.7  5.0
2025-01-03  15.7   8.3  6.1
2025-01-04  15.2  10.6  6.1
2025-01-05   2.0  12.8  8.9


C:\Users\brosb\AppData\Local\Temp\ipykernel_55876\347299688.py:48: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  df = raw_df.pivot_table(index='date', columns=


In [7]:
df.shape

(31, 3)